### Experiments: Comparing RSAST with SAST Ridge, HIVE-COTE, ROCKET, TS-CHIEF

In [1]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
from sktime.transformations.panel.shapelet_transform import ShapeletTransform
from cd_diagram.cd_function import *

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


In [2]:
ds_sens = tsc_dataset_names.univariate_equal_length
#ds_sens = ['ACSF1', 'Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown', 'ChlorineConcentration', 'CinCECGTorso', 'Coffee']
ds_sens = ds_sens[25:35]
max_ds=len(ds_sens) #exploring dataset in UEA & UCR Time Series Classification Repository
#print(ds_sens)
print(ds_sens)
print(max_ds)


['ECGFiveDays', 'ElectricDevices', 'EOGHorizontalSignal', 'EOGVerticalSignal', 'EthanolLevel', 'FaceAll', 'FaceFour', 'FacesUCR', 'FiftyWords', 'Fish']
10


In [3]:
runs = 1
#list
not_found_ds =[]
df_result = {}
list_score = []
list_time = []
list_dataset = []
list_hyperparameter = []
list_method = []
list_rpoint = []
list_nb_per_class = []


for ds in ds_sens:
    
    try:
        
        X_train, y_train = load_UCR_UEA_dataset(name=ds, extract_path='data', split="train", return_type="numpy2d")
        X_test, y_test = load_UCR_UEA_dataset(name=ds, extract_path='data', split="test", return_type="numpy2d")
        print("ds="+ds)
    except:
        print("not found ds="+ds)
        not_found_ds.append(ds)
        continue

    for i in range(runs):
        
        p=1
        k=10
        print("RSAST ACF&PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(k))
        start = time.time()
        random_state = None
        rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=k, len_method="both", classifier=RidgeClassifierCV())
        rsast_ridge.fit(X_train, y_train)
        end = time.time()
        list_score.append(rsast_ridge.score(X_test,y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("ACF&PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(k))
        list_rpoint.append(str(p))
        list_nb_per_class.append(str(k))
        list_method.append("ACF&PACF: Rsast")
        
        
        p=1
        k=10
        print("RSAST Max PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(k))
        start = time.time()
        random_state = None
        rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=k, len_method="Max PACF", classifier=RidgeClassifierCV())
        rsast_ridge.fit(X_train, y_train)
        end = time.time()
        list_score.append(rsast_ridge.score(X_test,y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("Max PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(k))
        list_rpoint.append(str(p))
        list_nb_per_class.append(str(k))
        list_method.append("Max PACF: Rsast")
        
        print("ROCKET: kernels=10_000")
        start = time.time()
        rocket= RocketClassifier(num_kernels=10_000)
        rocket.fit(X_train,y_train)
        end = time.time()
        list_score.append(rocket.score(X_test,y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("num_kernels=10_000")
        list_rpoint.append(str(0))
        list_nb_per_class.append(str(0))
        list_method.append("Rocket")
        '''
        min_shp_length = 3
        print("STC: min_shapelet_length="+str(min_shp_length)+" max_shapelet_length="+str(np.inf))
        
        max_shp_length = X_train.shape[1]
        start = time.time()
        #Creating a ShapeletTransform and fitting
        st = ShapeletTransform(min_shapelet_length=min_shp_length, max_shapelet_length=np.inf)
        X_train_sktime = from_2d_array_to_nested(pd.DataFrame(X_train))
        X_test_sktime = from_2d_array_to_nested(pd.DataFrame(X_test))
        st.fit(X_train_sktime, y_train)
        #Making the tranformation of time series and training a RandomForestClassifier model
        X_test_transformed = st.transform(X_test_sktime)
        X_train_transformed = st.transform(X_train_sktime)
        clf = RandomForestClassifier()
        clf.fit(X_train_transformed, y_train)
        
        end = time.time()
        list_score.append(clf.score(X_test_transformed, y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("min_shapelet_length="+str(min_shp_length)+" max_shapelet_length="+str(max_shp_length))
        list_rpoint.append(str(0))
        list_nb_per_class.append(str(0))
        list_method.append("STC")  
        '''
        '''
        min_shp_length = 3
        max_shp_length = X_train.shape[1]
        print("STC: min_shapelet_length="+str(min_shp_length)+" max_shapelet_length="+str(X_train.shape[1]))
        start = time.time()
        
        candidate_lengths = np.arange(min_shp_length, max_shp_length+1)
        random_state = None
        sast_ridge = SAST(cand_length_list=candidate_lengths,
                                nb_inst_per_class=1, 
                                random_state=random_state, classifier=RidgeClassifierCV())
        sast_ridge.fit(X_train, y_train)
        end = time.time()    
        list_score.append(sast_ridge.score(X_test, y_test))
        list_time.append(end-start)
        list_dataset.append(ds)
        list_hyperparameter.append("min_shapelet_length="+str(min_shp_length)+" max_shapelet_length="+str(max_shp_length))
        list_rpoint.append(str(0))
        list_nb_per_class.append(str(0))
        list_method.append("SAST")  
        '''
        
 

df_result['accuracy']=list_score
df_result['time']=list_time
df_result['dataset_name']=list_dataset
df_result['hyperparameter']=list_hyperparameter
df_result['rpoint']=list_rpoint
df_result['nb_per_class']=list_nb_per_class
df_result['classifier_name']=list_method
df_result=pd.DataFrame(df_result)
# export a overall dataset with the comparison
df_result.to_csv("df_overall_comparison_results.csv")

ds=ECGFiveDays
RSAST ACF&PACF: n_random_points=1 nb_inst_per_class=10
RSAST Max PACF: n_random_points=1 nb_inst_per_class=10
ROCKET: kernels=10_000
ds=ElectricDevices
RSAST ACF&PACF: n_random_points=1 nb_inst_per_class=10
RSAST Max PACF: n_random_points=1 nb_inst_per_class=10
ROCKET: kernels=10_000
ds=EOGHorizontalSignal
RSAST ACF&PACF: n_random_points=1 nb_inst_per_class=10
RSAST Max PACF: n_random_points=1 nb_inst_per_class=10
ROCKET: kernels=10_000
ds=EOGVerticalSignal
RSAST ACF&PACF: n_random_points=1 nb_inst_per_class=10
RSAST Max PACF: n_random_points=1 nb_inst_per_class=10


In [51]:
df_result=pd.read_csv("df_overall_comparison_results.csv")

In [20]:
'''
# create and export a pivot table with the mean of score of different methods
pivot_table = pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
pivot_table.to_csv("df_pivot_overall_comparison.csv") 
'''
df_result=df_result.rename(columns={'classifier_name':'classifier_name2'})
df_result=df_result.rename(columns={'method':'classifier_name'})

In [21]:
# create a pivot table with the mean of score by hyperparameter
pd.pivot_table(df_result, values='accuracy', index=['classifier_name','classifier_name2'],columns=['dataset_name'], aggfunc='mean')

,dataset_name,CricketY,CricketZ,Crop,DiatomSizeReduction,DistalPhalanxOutlineAgeGroup,DistalPhalanxOutlineCorrect,DistalPhalanxTW,ECG200,ECG5000,Earthquakes
classifier_name,classifier_name2,,,,,,,,,,
ACF&PACF: Rsast,ACF&PACF: n_random_points=1 nb_inst_per_class=10,0.746154,0.766667,0.683214,0.967320,0.762590,0.739130,0.661871,0.82,0.929556,0.604317
Max PACF: Rsast,Max PACF: n_random_points=1 nb_inst_per_class=10,0.658974,0.671795,0.614167,0.957516,0.755396,0.673913,0.683453,0.91,0.940889,0.748201
Rocket,num_kernels=10_000,0.843590,0.853846,0.758929,0.967320,0.741007,0.760870,0.690647,0.91,0.948444,0.755396


In [22]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result, values='accuracy', index=['method','classifier_name'],columns=['dataset_name'], aggfunc='var')


KeyError: 'method'

In [24]:
df_result

,accuracy,time,dataset_name,classifier_name2,rpoint,nb_per_class,classifier_name
0,0.746154,155.860067,CricketY,ACF&PACF: n_random_points=1 nb_inst_per_class=10,1,10,ACF&PACF: Rsast
1,0.658974,40.186798,CricketY,Max PACF: n_random_points=1 nb_inst_per_class=10,1,10,Max PACF: Rsast
2,0.843590,37.508186,CricketY,num_kernels=10_000,0,0,Rocket
3,0.766667,155.500592,CricketZ,ACF&PACF: n_random_points=1 nb_inst_per_class=10,1,10,ACF&PACF: Rsast
4,0.671795,42.552719,CricketZ,Max PACF: n_random_points=1 nb_inst_per_class=10,1,10,Max PACF: Rsast
5,0.853846,37.327309,CricketZ,num_kernels=10_000,0,0,Rocket
6,0.683214,98.091156,Crop,ACF&PACF: n_random_points=1 nb_inst_per_class=10,1,10,ACF&PACF: Rsast
7,0.614167,15.975322,Crop,Max PACF: n_random_points=1 nb_inst_per_class=10,1,10,Max PACF: Rsast
8,0.758929,171.315668,Crop,num_kernels=10_000,0,0,Rocket
9,0.967320,6.434163,DiatomSizeReduction,ACF&PACF: n_random_points=1 nb_inst_per_class=10,1,10,ACF&PACF: Rsast


In [23]:
# create a pivot table with the mean of time spent in minutes

pd.pivot_table(df_result, values='time', index=['classifier_name','classifier_name2'],columns=['dataset_name'], aggfunc='mean')

,dataset_name,CricketY,CricketZ,Crop,DiatomSizeReduction,DistalPhalanxOutlineAgeGroup,DistalPhalanxOutlineCorrect,DistalPhalanxTW,ECG200,ECG5000,Earthquakes
classifier_name,classifier_name2,,,,,,,,,,
ACF&PACF: Rsast,ACF&PACF: n_random_points=1 nb_inst_per_class=10,155.860067,155.500592,98.091156,6.434163,2.724563,2.576569,5.382552,1.029314,18.885293,65.428511
Max PACF: Rsast,Max PACF: n_random_points=1 nb_inst_per_class=10,40.186798,42.552719,15.975322,5.015217,0.327147,0.336138,0.667876,0.239985,1.491275,28.055224
Rocket,num_kernels=10_000,37.508186,37.327309,171.315668,1.659974,9.955172,14.310674,9.819463,3.183177,22.197425,52.294904


In [25]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
draw_cd_diagram(df_perf, labels=True, title="Overall comparison", name_file='cd-diagram_overall_comparison.png')
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)

/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families were found: Arial
findfont: Generic family 'sans-serif' not found because none of the following families 

['ACF&PACF: Rsast' 'Max PACF: Rsast' 'Rocket']
ACF&PACF: Rsast    1.0
Max PACF: Rsast    0.0
Rocket             7.0
dtype: float64
Max PACF: Rsast    2.45
ACF&PACF: Rsast    2.25
Rocket             1.30
dtype: float64
Index(['Max PACF: Rsast', 'ACF&PACF: Rsast', 'Rocket'], dtype='object')
[0, 1]
[0, 2]
['ACF&PACF: Rsast' 'Max PACF: Rsast' 'Rocket']
ACF&PACF: Rsast    1.0
Max PACF: Rsast    0.0
Rocket             7.0
dtype: float64


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/scipy/stats/_morestats.py:3414: UserWarning: Exact p-value calculation does not work if there are zeros. Switching to normal approximation.
  warnings.warn("Exact p-value calculation does not work if there are "


In [26]:
average_ranks

,0
Max PACF: Rsast,2.45
ACF&PACF: Rsast,2.25
Rocket,1.30
